In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings(action='ignore') 

from fbprophet.diagnostics import cross_validation
from statsmodels.tsa.arima_model import ARIMA
import statsmodels.api as sm
from statsmodels.tsa.api import ExponentialSmoothing, SimpleExpSmoothing, Holt
import numpy as np
from fbprophet import Prophet

In [2]:
train = pd.read_csv('01_음식물쓰레기_FOOD_WASTE_210811_update.CSV', encoding = 'cp949')
train

,base_date,city,emd_cd,emd_nm,em_area_cd,em_cnt,em_g,pay_amt
0,2018-01-01,제주시,50110670,외도동,W6X062,2,15500,464
1,2018-01-01,제주시,50110630,아라동,W6XA97,25,59800,1787
2,2018-01-01,제주시,50110630,아라동,W6XA96,8,25350,758
3,2018-01-01,제주시,50110630,아라동,W6XA95,12,30000,898
4,2018-01-01,제주시,50110630,아라동,W6XA94,14,44050,1317
...,...,...,...,...,...,...,...,...
2211440,2021-06-30,제주시,50110250,한림읍,W6XEB4,12,58250,1743
2211441,2021-06-30,제주시,50110250,한림읍,W6XEB7,15,111000,3326
2211442,2021-06-30,제주시,50110250,한림읍,W6XEB8,21,48300,1444
2211443,2021-06-30,제주시,50110250,한림읍,W6XEA2,5,22050,661


In [3]:
train1 = train.drop(columns=['city','emd_cd','em_area_cd','em_cnt','pay_amt'])
train1

,base_date,emd_nm,em_g
0,2018-01-01,외도동,15500
1,2018-01-01,아라동,59800
2,2018-01-01,아라동,25350
3,2018-01-01,아라동,30000
4,2018-01-01,아라동,44050
...,...,...,...
2211440,2021-06-30,한림읍,58250
2211441,2021-06-30,한림읍,111000
2211442,2021-06-30,한림읍,48300
2211443,2021-06-30,한림읍,22050


In [4]:
train1['base_date'] = pd.to_datetime(train1['base_date'])
train1['month'] = train1['base_date'].dt.month
train1['year'] = train1['base_date'].dt.year
train1['day'] = train1['base_date'].dt.day

In [5]:
train2 = train1.groupby(['year','month','day','emd_nm']).agg(sum).unstack()

In [6]:
train3 = train2.reset_index()

In [7]:
train3.columns = ['year',	'month',	'day',
                  '건입동',	'구좌읍',	'남원읍',	'노형동',	'대륜동',	'대정읍',	'대천동',	'도두동',	'동홍동',	'봉개동',	'삼도1동',	'삼도2동',	'삼양동',	'서홍동',	'성산읍',	'송산동',	
                  '아라동',	'안덕면',	'알수없음',	'애월읍',	'연동',	'영천동',	'예래동',	'오라동',	
                  '외도동',	'용담1동',	'용담2동',	'이도1동',	'이도2동',	'이호동',	'일도1동',	'일도2동',	'정방동',	'조천읍', '중문동',	'중앙동',	'천지동',	'표선면',	'한경면',	'한림읍',	'화북동',	'효돈동']

In [8]:
train3['Date'] = train3['year'].astype(str)+ '-' + train3['month'].astype(str)+ '-' + train3['day'].astype(str)

In [9]:
train3['Date'] = pd.to_datetime(train3['Date'])

In [10]:
train4 = train3.drop(columns=['year','month','day'])

In [11]:
p = Prophet()

In [12]:
sub = pd.DataFrame()
sub['DateTime'] = pd.date_range('2021-07-01','2021-08-31')

In [13]:
for col in train4.columns[:-1].tolist():
  train5 = train4[['Date', col]]
  train5.columns = ['ds','y']

  model = Prophet()
  model.fit(train5)

  future = model.make_future_dataframe(periods =365)
  fcst = model.predict(future)
#   print(f'{col} Forecasting....')

  result = fcst.iloc[1276:]
  
  result.rename(columns={'ds':'DateTime'}, inplace = True)
  
  # display(result)
  temp = result.groupby('DateTime')['yhat'].sum().reset_index(name = col)

  sub = pd.merge(sub,temp, on = 'DateTime', how = 'left')

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=T

In [14]:
sub['year'] = sub['DateTime'].dt.year
sub['month'] = sub['DateTime'].dt.month

In [15]:
sub_prophet = sub.groupby(['year','month']).agg(sum).reset_index()

In [16]:
sub2 = pd.DataFrame()
sub2['DateTime'] = pd.date_range('2021-07-01','2021-08-31')

In [17]:
df = train4[['Date','건입동']]
df.columns = ['ds','y']

In [18]:
model = Prophet()
model.fit(df)

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


In [19]:
future = model.make_future_dataframe(periods = 62, freq = 'D')
fcst = model.predict(future)
pred_df = fcst.iloc[1277:]

In [20]:
predict_ls = []
for col in train4.columns[:-1]:
  fit = sm.tsa.statespace.SARIMAX(train4[col], order = (1,1,1), seasonal_order=(1,1,1,12)).fit()
  predict = fit.predict(start = 1276, end = 1276+61, dynamic = True).tolist()
  predict_ls.append(predict)
#   print(f'{col} Forecasting....')

  temp = pd.DataFrame()
  temp['DateTime'] = pred_df['ds']
  temp.index = range(len(temp))

  temp[col] = predict
  temp2 = temp.groupby('DateTime')[col].sum().reset_index(name = col)

  sub2 = pd.merge(sub2, temp2, on='DateTime', how = 'left')

In [21]:
sub2['year'] = sub2['DateTime'].dt.year
sub2['month'] = sub2['DateTime'].dt.month

In [22]:
sub_arima = sub2.groupby(['year','month']).agg(sum).reset_index()

In [23]:
sub_pro_ari = sub_prophet.iloc[:,2:] *0.5 + sub_arima.iloc[:,2:]*0.5

In [26]:
a = sub_pro_ari.apply(lambda x : round(x,1), axis = 1)

In [27]:
a.to_csv('final_sub.csv', index = False)